In [1]:
import numpy as np
import pandas as pd

import yaml

In [2]:
with open("Config/config_file.yml", "r") as ymlfile:
    cfg = yaml.safe_load(ymlfile)

solution_to_analize = cfg["solution_to_analize"]
cost_euro_x_Km = cfg["cost_euro_x_Km"]

df = pd.read_excel("Results/"+solution_to_analize, index_col=0)
df_vehicles = pd.read_excel('Data/real_data.xlsx',sheet_name='Vehicle',index_col=0)


Metriche calcolate:

- veicoli utilizzati per ogni autocentro
- quantità e numero di servizi conferiti a ogni facility
- quantità media conferita a ogni facility
- % di facility che coincide con la preferred
- quantità media trasportata per tipologia di veicolo

In [4]:
##   We need df_depots for the following computations
df_depots = pd.read_excel('Data/real_data.xlsx',sheet_name='Depot',index_col=0)
df_depots.fillna(0,inplace=True)
vehicle_per_depot = df_depots[["vehicle"]]
vehicles_per_depot_list = []
for depot_id in vehicle_per_depot.index:
    depot_dic = eval(vehicle_per_depot[vehicle_per_depot.index==depot_id]["vehicle"][depot_id])
    for key in depot_dic.keys():
        template_row = [depot_id, key, depot_dic[key]]
        vehicles_per_depot_list.append(template_row)
df_vehicle_per_depot = pd.DataFrame(vehicles_per_depot_list, columns = ["vehicle_autocenter", "vehicle_type", "available"])
df_vehicle_per_depot.head(5)


,vehicle_autocenter,vehicle_type,available
0,AUTO_AC,COMP 2A,14
1,AUTO_AC,COMP 3A,14
2,AUTO_AC,COMP SIDE LOA 2A,2
3,AUTO_AC,COMP SIDE LOA 3A,21
4,AUTO_PM,COMP 2A,35


## Kilometers

In [5]:
df_metrics = pd.read_excel("Results/"+solution_to_analize, sheet_name="Metrics", index_col=0)
print("the objective function value is: ", df_metrics.values[0][0])
print("the number of kilometers travelled by the vehicles is: ", np.round(df_metrics.values[0][0]/cost_euro_x_Km,2))

the objective function value is:  15103.82
the number of kilometers travelled by the vehicles is:  16782.02


## Vehicle usage

In [6]:
# - df_vehicle_per_depot_used: vehicle departed by each autocenter - vehicle_type detail
df_vehicle_per_depot_used = df.groupby(["vehicle_autocenter", "vehicle_type"], as_index=False).size().rename(columns = {'size': "used"})
df_vehicle_per_depot_used = df_vehicle_per_depot.merge(df_vehicle_per_depot_used, on=["vehicle_autocenter", "vehicle_type"], how = 'outer').fillna(0)
df_vehicle_per_depot_used["%"] = np.round((df_vehicle_per_depot_used["used"]/df_vehicle_per_depot_used["available"])*100,1)
df_vehicle_per_depot_used = df_vehicle_per_depot_used[["vehicle_autocenter", "vehicle_type", "used", "available", "%"]]
df_vehicle_per_depot_used.head(10)
# - df_vehicle_per_depot_used_agg: vehicle departed by each autocenter - aggregated
df_vehicle_per_depot_used_agg = df.groupby(["vehicle_autocenter"], as_index=False).size().rename(columns = {'size': "used"})
df_vehicle_per_depot_used_agg = pd.merge(df_vehicle_per_depot_used_agg, df_vehicle_per_depot.groupby("vehicle_autocenter")[['available']].agg(sum).reset_index())
df_vehicle_per_depot_used_agg["%"] = (df_vehicle_per_depot_used_agg["used"]/df_vehicle_per_depot_used_agg["available"]*100).round(1)
df_vehicle_per_depot_used_agg.head()

,vehicle_autocenter,used,available,%
0,AUTO_AC,43,51,84.3
1,AUTO_PM,111,114,97.4
2,AUTO_RC,145,145,100.0
3,AUTO_SA,68,109,62.4
4,AUTO_TP,97,102,95.1


In [8]:
## df_vehicle_qnt_kg_per_type_agg: kg and number of services per vehicle type - aggregated
df_vehicle_qnt_kg_per_type_agg = df.groupby("vehicle_type").agg(s_count = ("service_id", "count"), mean_kg_per_s = ("service_quantity(kg)", "mean"), total_kg = ("service_quantity(kg)", "sum")).round(1)
df_vehicle_qnt_kg_per_type_agg = df_vehicle_qnt_kg_per_type_agg.join(df_vehicle_per_depot.groupby("vehicle_type")[["available"]].sum())
df_vehicle_qnt_kg_per_type_agg["%"] = np.round(df_vehicle_qnt_kg_per_type_agg["s_count"]/df_vehicle_qnt_kg_per_type_agg["available"],2)*100
df_vehicle_qnt_kg_per_type_agg = df_vehicle_qnt_kg_per_type_agg[["s_count","available","%", "mean_kg_per_s", "total_kg"]]

## df_vehicle_qnt_kg_per_type: kg and number of services per vehicle type - vehicle_type detail
df_vehicle_qnt_kg_per_type = df.groupby(["vehicle_autocenter","vehicle_type"]).agg(s_count = ("service_id", "count"), mean_kg_per_s = ("service_quantity(kg)", "mean"), total_kg = ("service_quantity(kg)", "sum")).round(1)
df_vehicle_qnt_kg_per_type = df_vehicle_qnt_kg_per_type.join(df_vehicle_per_depot.groupby(["vehicle_autocenter","vehicle_type"])[["available"]].sum())
df_vehicle_qnt_kg_per_type["%"] = np.round(df_vehicle_qnt_kg_per_type["s_count"]/df_vehicle_qnt_kg_per_type["available"],2)*100
df_vehicle_qnt_kg_per_type = df_vehicle_qnt_kg_per_type[["s_count","available","%", "mean_kg_per_s", "total_kg"]]
df_vehicle_qnt_kg_per_type

s_count  available      %  mean_kg_per_s  \
vehicle_autocenter vehicle_type                                                 
AUTO_AC            COMP 2A                 7         14   50.0         2554.3   
                   COMP 3A                14         14  100.0         7050.7   
                   COMP SIDE LOA 2A        1          2   50.0         3780.0   
                   COMP SIDE LOA 3A       21         21  100.0         6386.7   
AUTO_PM            COMP 2A                32         35   91.0         2457.8   
                   COMP 3A                16         16  100.0         7376.2   
                   COMP SIDE LOA 2A       17         17  100.0         3072.4   
                   COMP SIDE LOA 3A       46         46  100.0         6781.5   
AUTO_RC            COMP 2A                31         31  100.0         2592.9   
                   COMP 3A                27         27  100.0         5947.0   
                   COMP SIDE LOA 2A       10         10  100.0         4068.0   
                   COMP SIDE LOA 3A       77         77  100.0         6284.9   
AUTO_SA            COMP 2A                21         32   66.0         2171.0   
                   COMP 3A                15         21   71.0         5055.3   
                   COMP SIDE LOA 2A        5          8   62.0         2816.0   
                   COMP SIDE LOA 3A       27         48   56.0         5020.0   
AUTO_TP            COMP 2A                24         29   83.0         2641.7   
                   COMP 3A                28         28  100.0         6606.1   
                   COMP SIDE LOA 2A        9          9  100.0         3986.7   
                   COMP SIDE LOA 3A       36         36  100.0         5985.0   

                                     total_kg  
vehicle_autocenter vehicle_type                
AUTO_AC            COMP 2A              17880  
                   COMP 3A              98710  
                   COMP SIDE LOA 2A      3780  
                   COMP SIDE LOA 3A    134120  
AUTO_PM            COMP 2A              78650  
                   COMP 3A             118020  
                   COMP SIDE LOA 2A     52230  
                   COMP SIDE LOA 3A    311950  
AUTO_RC            COMP 2A              80380  
                   COMP 3A             160570  
                   COMP SIDE LOA 2A     40680  
                   COMP SIDE LOA 3A    483940  
AUTO_SA            COMP 2A              45590  
                   COMP 3A              75830  
                   COMP SIDE LOA 2A     14080  
                   COMP SIDE LOA 3A    135540  
AUTO_TP            COMP 2A              63400  
                   COMP 3A             184970  
                   COMP SIDE LOA 2A     35880  
                   COMP SIDE LOA 3A    215460

## Facility usage

In [9]:
## df_facility_results: kg and number of services per facility
df_facility_real = pd.read_excel("Data/real_data.xlsx", sheet_name= "Facility", index_col=0)
df_facility_results = df.groupby("facility_id").agg(s_count = ("service_id", "count"), mean_kg_per_s = ("service_quantity(kg)", "mean"), total_kg = ("service_quantity(kg)", "sum")).round(1)
df_facility_results.head(2)

,s_count,mean_kg_per_s,total_kg
facility_id,,,
IMP_000003_0012,29,3035.9,88040
IMP_000004_0011,11,5962.7,65590


In [10]:
df_facility_summary = df_facility_real[["capacity(kg)", "material"]].join(df_facility_results)
df_facility_summary = df_facility_summary[["material", "s_count", "mean_kg_per_s", "total_kg", "capacity(kg)"]].fillna(0)
df_facility_summary["%"] = (df_facility_summary["total_kg"]/df_facility_summary["capacity(kg)"]*100).round(2)
df_facility_summary

,material,s_count,mean_kg_per_s,total_kg,capacity(kg),%
facilty_id,,,,,,
IMP_000166_0014,rifiuto rd umido,57.0,6307.4,359520.0,300000000,0.12
IMP_000018_0013,rifiuto rd carta,0.0,0.0,0.0,300000000,0.00
IMP_000004_0011,rifiuto indifferenziato,11.0,5962.7,65590.0,300000000,0.02
IMP_000024_0014,rifiuto rd umido,0.0,0.0,0.0,300000000,0.00
IMP_000179_0013,rifiuto rd carta,0.0,0.0,0.0,300000000,0.00
IMP_999107_0011,rifiuto indifferenziato,3.0,4316.7,12950.0,300000000,0.00
IMP_000189_0011,rifiuto indifferenziato,37.0,5970.0,220890.0,300000000,0.07
IMP_000176_0013,rifiuto rd carta,1.0,3780.0,3780.0,300000000,0.00
IMP_000009_0012,rifiuto rd multimateriale,2.0,3740.0,7480.0,300000000,0.00


## Comparison with AMA

In [11]:
df_preferred_facility = pd.DataFrame(df, columns =["service_id", "facility_id", "preferred_facility_id", "preferred_facility_desc"])
df_preferred_facility["is_preferred"] = 0
df_preferred_facility.loc[df_preferred_facility["facility_id"] == df_preferred_facility["preferred_facility_id"], 'is_preferred'] = 1

## df_preferred_facility_per_s: facilities that coincide with the preferred one grouped by facilities
df_preferred_facility_per_s = df_preferred_facility.groupby("preferred_facility_desc").agg(s_count = ("service_id", "count"), same_facility_prefer = ("is_preferred", "sum"))
df_preferred_facility_per_s["%"] = ((df_preferred_facility_per_s["same_facility_prefer"]/df_preferred_facility_per_s["s_count"])*100).round(1)
df_preferred_facility_per_s

,s_count,same_facility_prefer,%
preferred_facility_desc,,,
A.I.A. LAURENTINA - stazionamento bilico organico (RIFIUTO RD UMIDO),10,10,100.0
BOX 3 S.R.L. (RIFIUTO RD CARTA),24,0,0.0
CDR RC (RIFIUTO INDIFFERENZIATO),103,8,7.8
COMPOSTAGGIO MACCARESE (RIFIUTO RD UMIDO),47,0,0.0
ECO LOGICA 2000 S.R.L. (RIFIUTO RD CARTA),4,0,0.0
INTERCARTA (RIFIUTO RD CARTA),23,0,0.0
MULTI LAURENTINA (RIFIUTO RD MULTIMATERIALE),12,2,16.7
MULTI RC (RIFIUTO RD MULTIMATERIALE),33,25,75.8
RICICLA ITALIA (RIFIUTO RD CARTA),16,14,87.5


In [12]:
## df_preferred_facility_per_s_agg: facilities that coincide with the preferred one - aggregated
df_preferred_facility_per_s_agg = pd.DataFrame({"s_count" : len(df_preferred_facility), "same_facility_prefer" : df_preferred_facility["is_preferred"].sum(), "%" : (df_preferred_facility["is_preferred"].sum()/len(df_preferred_facility)*100).round(1)}, index=["total"])
df_preferred_facility_per_s_agg

,s_count,same_facility_prefer,%
total,464,146,31.5
